# Laplacian Convolutional Representation

In [1]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def prox_2d(z, w, lmbda, denominator):
    T = z.shape[0]
    temp1 = np.fft.fft(lmbda * z - w) / denominator
    temp2 = 1 - T / (denominator * np.abs(temp1))
    temp2[temp2 <= 0] = 0
    return np.fft.ifft(temp1 * temp2).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def flip_mat(X):
    return np.append(X, np.flip(X, axis = 1), axis = 1)

def inv_flip_mat(mat):
    dim = mat.shape
    T = int(dim[1] / 2)
    return (mat[:, : T] + np.flip(mat[:, T :], axis = 1)) / 2

def LCR_3d(y_true, y, lmbda, gamma, tau_t, maxiter = 50):
    dim1, dim2, dim3 = y.shape
    eta = 100 * lmbda
    if np.isnan(y).any() == False:
        pos_test = np.where((y_true != 0) & (y == 0))
    elif np.isnan(y).any() == True:
        pos_test = np.where((y_true > 0) & (np.isnan(y)))
        y[np.isnan(y)] = 0
    y_test = y_true[pos_test]
    flip_y_true = np.zeros((dim1, 2 * dim2, dim3))
    flip_y = np.zeros((dim1, 2 * dim2, dim3))
    for t in range(dim3):
        flip_y_true[:, :, t] = flip_mat(y_true[:, :, t])
        flip_y[:, :, t] = flip_mat(y[:, :, t])
    N, T, L = flip_y.shape
    pos_train = np.where(flip_y != 0)
    y_train = flip_y[pos_train]
    z = flip_y.copy()
    w = flip_y.copy()
    ell_t = laplacian(T, tau_t)
    ell = np.fft.fft(ell_t)
    denominator = lmbda + gamma * np.abs(ell) ** 2
    del y_true, y
    show_iter = 10
    for it in range(maxiter):
        x = np.zeros((N, T, L))
        for t in range(dim3):
            for n in range(dim1):
                x[n, :, t] = prox_2d(z[n, :, t], w[n, :, t], lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        y_hat = np.zeros((dim1, dim2, dim3))
        for t in range(dim3):
            y_hat[:, :, t] = inv_flip_mat(x[:, :, t])
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, y_hat[pos_test]))
            print(compute_rmse(y_test, y_hat[pos_test]))
            print()
    return y_hat

## HighD Dataset

Hyperparameters:

- $\lambda=10^{-3}T$
- $\gamma = 5\lambda$
- $\tau=3$

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

def plot_speed_field(data, filename, x_scale=2, y_scale=3):
    fig = plt.figure(figsize = (2.5*2.5, 5))
    x = range(data.shape[1])
    x_new = [i * x_scale for i in x]
    y = range(data.shape[0])
    y_new = [i * y_scale for i in y]
    plt.matshow(data, cmap='jet_r', origin='lower', 
                extent=[x_new[0], x_new[-1], y_new[0], y_new[-1]],
                vmin = 5, vmax = 35, fignum = 1)
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (m/s)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

dense_tensor = np.load('../datasets/HighD/speed_matrix_full_46.npy').transpose(1, 0, 2)
sparse_tensor = np.load('../datasets/HighD/speed_matrix_70_46.npy').transpose(1, 0, 2)

# for lane in range(3):
#     plot_speed_field(dense_tensor[:, :, lane],
#                      'speed_matrix_full_46_lane{}.png'.format(lane + 1))
#     plot_speed_field(sparse_tensor[:, :, lane],
#                      'speed_matrix_70_46_lane{}.png'.format(lane + 1))

import time
start = time.time()
N, T, L = sparse_tensor.shape
lmbda = 1e-3 * T
gamma = 5 * lmbda
tau_t = 3
maxiter = 100
tensor_hat = LCR_3d(dense_tensor, sparse_tensor, lmbda, gamma, tau_t, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

# for lane in range(3):
#     plot_speed_field(tensor_hat[:, :, lane], 
#                      'HighD_speed_field_70_LCR_rec_lane{}.png'.format(lane + 1))

<ipython-input-1-4f393bce6ff2>:20: RuntimeWarning: divide by zero encountered in true_divide
  temp2 = 1 - T / (denominator * np.abs(temp1))


10
0.2618282590885133
6.940408515918634

20
0.1226685372858289
4.3714615842035816

30
0.09260498189242067
3.7196623200156798

40
0.08499456400839996
3.536793603279931

50
0.08215519356003377
3.445650244292017

60
0.08183485542156861
3.479732507009396

70
0.08162259730193608
3.495992947227681

80
0.08058277085521759
3.425768956476862

90
0.07960021672640455
3.3578345804829186

100
0.07907129880553354
3.3194628880483155

Running time: 4 seconds.


## CitySim Dataset

Hyperparameters:

- $\lambda=10^{-2}T$
- $\gamma = 5\lambda$
- $\tau=3$

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

def plot_speed_field(data, filename, x_scale=2, y_scale=5):
    fig = plt.figure(figsize = (2.3*2.3, 1.8))
    x = range(data.shape[1])
    x_new = [i * x_scale for i in x]
    y = range(data.shape[0])
    y_new = [i * y_scale for i in y]
    plt.matshow(data, cmap='jet_r', origin='lower',
                extent=[x_new[0], x_new[-1], y_new[0], y_new[-1]],
                vmin = 14, vmax = 35, fignum = 1, aspect='auto')
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (m/s)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

dense_tensor = np.load('../datasets/CitySim/speed_matrix_full.npy').transpose(1, 0, 2)
sparse_tensor = np.load('../datasets/CitySim/speed_matrix_70_FB.npy').transpose(1, 0, 2)

# for lane in range(3):
#     plot_speed_field(dense_tensor[:, :, lane],
#                      'speed_matrix_full_lane{}.png'.format(lane + 1))
#     plot_speed_field(sparse_tensor[:, :, lane],
#                      'speed_matrix_70_lane{}.png'.format(lane + 1))

import time
start = time.time()
N, T, L = sparse_tensor.shape
lmbda = 1e-2 * T
gamma = 1 * lmbda
tau_t = 3
maxiter = 100
tensor_hat = LCR_3d(dense_tensor, sparse_tensor, lmbda, gamma, tau_t, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

# for lane in range(3):
#     plot_speed_field(tensor_hat[:, :, lane], 
#                      'CitySim_speed_field_70_LCR_rec_lane{}.png'.format(lane + 1))

<ipython-input-1-4f393bce6ff2>:20: RuntimeWarning: divide by zero encountered in true_divide
  temp2 = 1 - T / (denominator * np.abs(temp1))


10
0.14147542306039557
4.4278360200996625

20
0.10667800122326916
3.7076646841203216

30
0.10352168827561954
3.674272780533107

40
0.10333226216357992
3.6729441967237424

50
0.10332541235371091
3.675614645239183

60
0.10331600887891852
3.676191032044014

70
0.1033090021748966
3.6766709856656594

80
0.10329573218477128
3.676764119636437

90
0.10328863056596149
3.6768305791903257

100
0.10328145791402017
3.6768022891462016

Running time: 3 seconds.


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>